## Deep Classification Network

0. First, the grid established in the Dataset exploration notebook will be tested with the SVM. Then, a DNN is implemented.

1. Import all `modules` we are going to need

In [1]:
import torch
from torch import nn
import torchvision

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

2. Import Input Matrix X. Then, import y

In [1]:
import os
import pandas as pd

In [2]:
os.chdir("/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis")

In [3]:
X = pd.read_csv('X_matrix.csv')

3. Import y

In [5]:
os.chdir('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/kosti/mat2csv/emotic_pre/')
train = pd.read_csv('train.csv')
train_mscoco = train[train['Folder'] == 'mscoco/images']

In [6]:
import re

In [7]:
y = []
i=0
for rating in train_mscoco['Continuous_Labels']:
    y.append(rating)

In [10]:
y_ = []
for idx, label in enumerate(y):
    s = (re.sub(r"[\[\]]",'',y[idx]))
    for i, x in enumerate(s):
        y_.append(x)
        if x == ',':
            y_.remove(x)
        if x == ' ':
            y_.remove(x)
        try:
            if x == '1' and s[i+1] == '0':
                y_.remove(x)
                y_.append('10')
        except IndexError:
            continue
        



In [12]:
for idx, el in enumerate(y_):
    if el == '10':
        if y_[idx+1] == '0':
            y_.remove(y_[idx+1])



In [13]:
len(y_)

42828

In [14]:
cont_labels = []
for val in y_:
    cont_labels.append(float(val))

In [15]:
img = 0
Arousal = []
Valence = []
Dominance = []
for val in cont_labels:
    if img == 0:
        Arousal.append(val)
        img = img+1
        continue
    if img == 1:
        Valence.append(val)
        img = img+1
        continue 
    if img == 2:
        Dominance.append(val)
        img = 0
        continue
    



In [18]:
import numpy as np

Arousal = np.array(Arousal)



In [19]:
Arousal.shape

(14276,)

In [24]:
y = Arousal

Transform X to numpy array

In [20]:
X = np.array(X)



14276

In [25]:
from sklearn.feature_selection import SelectKBest



In [ ]:
X_new = SelectKBest(k=40000).fit_transform(X, y)

